In [2]:
import yfinance as yf
import pandas as pd
import numpy as np

In [3]:
tickers = ['XLK', 'XLV', 'XLF', 'XLE', 'XLP', 'XLI', 'XLC', 'XLB', 'XLU', 'XLRE', 'XLY']
start_date = '2015-01-01'
end_date = '2025-01-01'

# Periodi eventi (Pandemia) (Guerra)
covid_start = pd.Timestamp('2020-01-10')
covid_end   = pd.Timestamp('2023-05-05')

war_start   = pd.Timestamp('2022-02-24')
war_end     = pd.Timestamp.today().normalize()  # fino ad oggi


In [4]:
war_end

Timestamp('2025-11-15 00:00:00')

In [8]:
dats = yf.download(tickers, period='10y', auto_adjust=True, group_by='column')


[*********************100%***********************]  11 of 11 completed


In [15]:
dats

Price           Close                                                \
Ticker            XLB         XLC        XLE        XLF         XLI   
Date                                                                  
2015-11-16  36.733620         NaN  45.398010  16.280190   45.278175   
2015-11-17  36.692783         NaN  44.898628  16.246565   45.186298   
2015-11-18  37.346249         NaN  45.637714  16.535721   45.812729   
2015-11-19  37.305405         NaN  45.025135  16.562620   46.013191   
2015-11-20  37.191048         NaN  44.545715  16.589518   46.247044   
...               ...         ...        ...        ...         ...   
2025-11-10  86.629997  113.279999  90.349998  52.970001  154.199997   
2025-11-11  87.529999  114.199997  91.529999  53.209999  154.240005   
2025-11-12  88.150002  113.540001  90.250000  53.669998  154.389999   
2025-11-13  87.570000  111.930000  90.480003  52.980000  152.000000   
2025-11-14  86.769997  111.330002  92.019997  52.450001  152.009995   

Price                                                                ...  \
Ticker             XLK        XLP       XLRE        XLU         XLV  ...   
Date                                                                 ...   
2015-11-16   38.303841  37.273273  20.935026  31.364584   59.638355  ...   
2015-11-17   38.312744  37.257957  21.261139  30.799072   59.892345  ...   
2015-11-18   38.891479  37.771282  21.512814  30.994823   61.060730  ...   
2015-11-19   39.069565  37.870872  21.629791  31.328314   60.036282  ...   
2015-11-20   39.354469  37.602734  21.807028  31.458849   60.552715  ...   
...                ...        ...        ...        ...         ...  ...   
2025-11-10  295.529999  76.430000  41.240002  89.660004  147.279999  ...   
2025-11-11  293.010010  77.430000  41.700001  89.709999  150.679993  ...   
2025-11-12  293.920013  77.320000  41.369999  89.919998  152.820007  ...   
2025-11-13  286.589996  77.320000  40.830002  88.680000  152.740005  ...   
2025-11-14  288.149994  77.260002  40.950001  88.760002  151.830002  ...   

Price          Volume                                                    \
Ticker            XLC       XLE       XLF       XLI       XLK       XLP   
Date                                                                      
2015-11-16        NaN  17363100  53560564  13303900   9079200   9734900   
2015-11-17        NaN  15228900  46013180  15564400   9394400   8056200   
2015-11-18        NaN  20951500  83572836  10434800  10166800   7826200   
2015-11-19        NaN  18912500  35613076  12257400   9367500   7421100   
2015-11-20        NaN  14214000  44189330   9014100   7288200   5487200   
...               ...       ...       ...       ...       ...       ...   
2025-11-10  4724100.0  15632300  42010600  10900400   9143200  16805900   
2025-11-11  3841900.0  19376400  29995800   8652400   7435800  19801200   
2025-11-12  4563800.0  14931600  46052700   9583300   6589600  10978200   
2025-11-13  6354300.0  16121900  50455300  15475400  11192900  15370800   
2025-11-14  4859800.0  17089700  46149000  13433100  13710800  13138200   

Price                                             
Ticker         XLRE       XLU       XLV      XLY  
Date                                              
2015-11-16     4200   9175600  12959800  7383200  
2015-11-17    25700  13494000   7882300  7452300  
2015-11-18    28600  15462500  12816300  7137600  
2015-11-19     3500  12364800  11128000  5685200  
2015-11-20      600  12520000   9224500  5564900  
...             ...       ...       ...      ...  
2025-11-10  6823600   8973600  12305900  5217400  
2025-11-11  5743700   7217700  12659500  6387700  
2025-11-12  5801500   7509200  17847500  4657100  
2025-11-13  8032500  10337700  21807600  7634200  
2025-11-14  6434600  10084100  15467900  7331100  

[2515 rows x 55 columns]

In [16]:
if isinstance(dats.columns, pd.MultiIndex):
    print('primo if')
    if 'Close' in dats.columns.get_level_values(0):
        print('siamo qua')
        close = dats['Close'].copy()

primo if
siamo qua


In [17]:
close

Ticker,XLB,XLC,XLE,XLF,XLI,XLK,XLP,XLRE,XLU,XLV,XLY
Date,,,,,,,,,,,
2015-11-16,36.733620,NaN,45.398010,16.280190,45.278175,38.303841,37.273273,20.935026,31.364584,59.638355,70.491058
2015-11-17,36.692783,NaN,44.898628,16.246565,45.186298,38.312744,37.257957,21.261139,30.799072,59.892345,70.625435
2015-11-18,37.346249,NaN,45.637714,16.535721,45.812729,38.891479,37.771282,21.512814,30.994823,61.060730,71.835098
2015-11-19,37.305405,NaN,45.025135,16.562620,46.013191,39.069565,37.870872,21.629791,31.328314,60.036282,71.924713
2015-11-20,37.191048,NaN,44.545715,16.589518,46.247044,39.354469,37.602734,21.807028,31.458849,60.552715,72.784904
...,...,...,...,...,...,...,...,...,...,...,...
2025-11-10,86.629997,113.279999,90.349998,52.970001,154.199997,295.529999,76.430000,41.240002,89.660004,147.279999,238.960007
2025-11-11,87.529999,114.199997,91.529999,53.209999,154.240005,293.010010,77.430000,41.700001,89.709999,150.679993,239.550003
2025-11-12,88.150002,113.540001,90.250000,53.669998,154.389999,293.920013,77.320000,41.369999,89.919998,152.820007,237.940002


In [21]:

_open = dats['Open'] if isinstance(dats.columns, pd.MultiIndex) else dats[['Open']]
_close = dats['Close'] if isinstance(dats.columns, pd.MultiIndex) else dats[['Close']]

Esempio colonne Open/close (testa):
Ticker            XLB  XLC        XLE        XLF        XLI        XLK  \
Date                                                                     
2015-11-16  36.341533  NaN  43.873201  16.038106  44.467998  37.662771   
2015-11-17  36.798974  NaN  45.278166  16.347434  45.336641  38.383976   
2015-11-18  36.823467  NaN  45.211567  16.340709  45.228066  38.490811   
2015-11-19  37.256396  NaN  45.371378  16.542447  45.704155  38.927105   
2015-11-20  37.395259  NaN  45.038447  16.609691  46.196931  39.229818   

Ticker            XLP       XLRE        XLU        XLV        XLY  
Date                                                               
2015-11-16  36.522442  20.935026  30.900568  58.774764  69.371015  
2015-11-17  37.372878  21.268228  31.350093  59.756884  70.840481  
2015-11-18  37.365223  21.013010  30.820816  59.926209  70.822582  
2015-11-19  37.794259  21.558897  31.067305  60.671273  71.888878  
2015-11-20  38.039440  21.771580  31.

In [31]:
px=close

In [23]:
def log_returns(px: pd.DataFrame) -> pd.DataFrame:
    px = px.sort_index()
    rets = np.log(px / px.shift(1))
    return rets.dropna(how='all')

In [32]:
rets = log_returns(px)

In [33]:
print(rets)

Ticker           XLB       XLC       XLE       XLF       XLI       XLK  \
Date                                                                     
2015-11-17 -0.001112       NaN -0.011061 -0.002068 -0.002031  0.000232   
2015-11-18  0.017652       NaN  0.016327  0.017641  0.013768  0.014993   
2015-11-19 -0.001094       NaN -0.013514  0.001625  0.004366  0.004569   
2015-11-20 -0.003070       NaN -0.010705  0.001623  0.005069  0.007266   
2015-11-23  0.001317       NaN  0.007149 -0.004062 -0.005069 -0.005900   
...              ...       ...       ...       ...       ...       ...   
2025-11-10  0.009394  0.010382  0.009006  0.003593  0.005658  0.025254   
2025-11-11  0.010335  0.008089  0.012976  0.004521  0.000259 -0.008564   
2025-11-12  0.007058 -0.005796 -0.014083  0.008608  0.000972  0.003101   
2025-11-13 -0.006601 -0.014282  0.002545 -0.012940 -0.015601 -0.025255   
2025-11-14 -0.009178 -0.005375  0.016877 -0.010054  0.000066  0.005429   

Ticker           XLP      XLRE       

In [41]:
def annualized_volatility(rets: pd.DataFrame, periods_per_year = 252) -> pd.Series:
    return rets.std(skipna=True) * np.sqrt(periods_per_year)

In [43]:
vol = annualized_volatility(rets)
print(vol)

Ticker
XLB     0.208484
XLC     0.227598
XLE     0.300740
XLF     0.224312
XLI     0.199397
XLK     0.242300
XLP     0.147207
XLRE    0.207790
XLU     0.192080
XLV     0.167124
XLY     0.220327
dtype: float64


In [44]:
def drawdown_series(px: pd.DataFrame) -> pd.DataFrame:
    cummax = px.cummax()
    dd = px / cummax - 1.0
    return dd

In [45]:
dd = drawdown_series(px)

In [46]:
print(dd)

Ticker           XLB       XLC       XLE       XLF       XLI       XLK  \
Date                                                                     
2015-11-16  0.000000       NaN  0.000000  0.000000  0.000000  0.000000   
2015-11-17 -0.001112       NaN -0.011000 -0.002065 -0.002029  0.000000   
2015-11-18  0.000000       NaN  0.000000  0.000000  0.000000  0.000000   
2015-11-19 -0.001094       NaN -0.013423  0.000000  0.000000  0.000000   
2015-11-20 -0.004156       NaN -0.023928  0.000000  0.000000  0.000000   
...              ...       ...       ...       ...       ...       ...   
2025-11-10 -0.094951 -0.048027 -0.040245 -0.021430 -0.010206 -0.028277   
2025-11-11 -0.085548 -0.040296 -0.027710 -0.016996 -0.009949 -0.036563   
2025-11-12 -0.079071 -0.045842 -0.041307 -0.008498 -0.008986 -0.033571   
2025-11-13 -0.085130 -0.059372 -0.038864 -0.021245 -0.024328 -0.057673   
2025-11-14 -0.093488 -0.064414 -0.022505 -0.031036 -0.024263 -0.052543   

Ticker           XLP      XLRE       

In [50]:
def max_drawdown(px: pd.DataFrame) -> pd.Series:
    return drawdown_series(px).min()

In [52]:
max_dd = max_drawdown(px)
print(max_dd)

Ticker
XLB    -0.372742
XLC    -0.466536
XLE    -0.668130
XLF    -0.428625
XLI    -0.423341
XLK    -0.335591
XLP    -0.245123
XLRE   -0.388248
XLU    -0.360668
XLV    -0.284043
XLY    -0.396696
dtype: float64
